This file is part of BrainMolecularAtlas.

This file performs flux variability analysis and is needed for the Figure 11A

Copyright (c) 2021 Blue Brain Project/EPFL 

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.
This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <https://www.gnu.org/licenses/>.

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
from gurobipy import *
from optlang.gurobi_interface import Model, Variable, Constraint, Objective

In [3]:
import cobra
from cobra.io import read_sbml_model

from cobra.flux_analysis import flux_variability_analysis
import escher

from cobra.flux_analysis import production_envelope

from cobra.util.solver import linear_reaction_coefficients

from cobra.flux_analysis import single_gene_deletion, single_reaction_deletion

from cobra.flux_analysis import find_essential_genes
from cobra import Model, Reaction, Metabolite

from cobra.flux_analysis.loopless import add_loopless, loopless_solution


In [4]:
from sklearn import preprocessing

In [5]:
# mouse metabolism model
model = read_sbml_model("../data/iMM1415.xml.gz")
model

Academic license - for non-commercial use only


Name,iMM1415
Memory address,0x07fb33fe5b198
Number of metabolites,2775
Number of reactions,3726
Number of groups,99
Objective expression,1.0*BIOMASS_mm_1_no_glygln - 1.0*BIOMASS_mm_1_no_glygln_reverse_754f5
Compartments,"cytosol, extracellular space, golgi apparatus, lysosome, mitochondria, nucleus, endoplasmic reticulum, peroxisome/glyoxysome"


In [6]:
model.solver # check solver

In [7]:
solution_ini = model.optimize()
solution_ini

,fluxes,reduced_costs
DM_13_cis_oretn_n,0.000000,-0.000000e+00
DM_13_cis_retn_n,0.000000,-0.000000e+00
DM_Asn_X_Ser_Thr_l,0.000000,0.000000e+00
DM_Ser_Gly_Ala_X_Gly_l,0.000000,0.000000e+00
DM_Ser_Thr_l,0.000000,0.000000e+00
...,...,...
YVITEt,0.000000,0.000000e+00
BIOMASS_mm_1_no_glygln,1.363428,2.942091e-15
R03184_PLUS_R00647,0.000000,-0.000000e+00
R01465m,0.473717,0.000000e+00


In [8]:
#getting all fluxes that are greater than 0.0001 mmol/g* DWh −1

solution_ini_frame = solution_ini.to_frame()
solution_ini_frame[solution_ini_frame.fluxes.abs() > 1e-4]

,fluxes,reduced_costs
EX_2mcit_e,0.307217,0.000000e+00
EX_acac_e,0.453265,0.000000e+00
EX_arg__L_e,-1.000000,-4.957920e-01
EX_co2_e,31.317570,0.000000e+00
EX_glc__D_e,-1.000000,-0.000000e+00
...,...,...
XYLTD_D,0.317542,0.000000e+00
XYLUR,0.317542,0.000000e+00
BIOMASS_mm_1_no_glygln,1.363428,2.942091e-15
R01465m,0.473717,0.000000e+00


In [9]:
result = flux_variability_analysis(model)
abs_flux_ranges = result.maximum.to_dict()
escher.Builder('RECON1.Glycolysis TCA PPP', reaction_data=abs_flux_ranges).display_in_notebook()

In [ ]:
loopless_result = flux_variability_analysis(model,fraction_of_optimum=0.9,loopless=True)

abs_flux_max_90optIniIni = loopless_result.maximum.to_dict()
map_sol_fl_ini = escher.Builder('RECON1.Glycolysis TCA PPP', reaction_data=abs_flux_max_90optIniIni)


In [11]:
map_sol_fl_ini.save_html('looplessFVA_max_iniModelIniObj_90opt_28july2020.html')


'looplessFVA_max_iniModelIniObj_90opt_28july2020.html'

##### Gene names for Bigg Gene IDs

In [12]:
geneIdsMapping = pd.read_csv('../data/iMM1415genesCurl12july2019.txt', sep='\t') # /Users/polina/git/bbpMolAtlas/iMM1415genesCurl12july2019.txt


geneIdsMapping['gene_name'] = geneIdsMapping['gene_name'].str.upper()
print(len(geneIdsMapping))
print(len(geneIdsMapping[geneIdsMapping['gene_name']!='NOTGIVEN'])/len(geneIdsMapping))
geneIdsMapping = geneIdsMapping[geneIdsMapping['gene_name']!='NOTGIVEN']
print(len(geneIdsMapping))
geneIdsMapping.head()

1375
0.9636363636363636
1325


,bigg_id,gene_name
1,100041375,CYP3A41B
5,100043684,AMY2A4
6,100163,PAFAH2
7,100198,H6PD
8,100559,UGT2B38


In [13]:
import pickle as pkl

with open('../data/6_df_processed.pkl','rb') as f:
    conc_df = pkl.load(f)
print(len(conc_df))

conc_df["log_conc_uM_medNorm"]  = conc_df["log_conc_uM_medNorm"].astype('float64')



custom_dict = {'Itzhak 2017':0,
               'Wisniewski 2015':1,
              'Bai 2020':2,
              'Hasan 2020':3,
              'Geiger 2013':4,
              'Kjell 2020':5,
              'Fecher 2019':6,
              'Guergues 2019':7,
              'Hamezah 2019':8,
              'McKetney 2019':9,
              'Hamezah 2018':10,
              'Krogager 2018':11,
              'Zhu 2018':12,
              'Carlyle 2017':13,
              'Sharma 2015, isolated':14,
              'Sharma 2015, cultured':15,
              'Han 2014':16,
              'Davis 2019':17,
              'Chuang 2018':18,
              'Fornasiero 2018':19,
              'Hosp 2017, insoluble':20,
              'Hosp 2017, soluble':21,
              'Hosp 2017, CSF':22,
              'Beltran 2016':23,
              'Duda 2018':24}   

conc_df = conc_df.iloc[conc_df['Study'].map(custom_dict).argsort()]


conc_df = conc_df.reset_index(drop=True)

conc_df['exp_conc'] = np.exp(conc_df['log_conc_uM_medNorm'])


2131244


In [14]:
conc_df.columns

Index(['gene_names', 'Uniprot', 'Study', 'Organism', 'location', 'Age_cat',
       'Age_days', 'condition', 'sample_id', 'molecular_weight_kDa',
       'raw_data', 'raw_data_units', 'gene_name_unified', 'Uniprot_unified',
       'gene_id_final', 'log_raw_data', 'uniprot_from_gn', 'Uniprot_final',
       'TheorPepNum', 'conc_uM', 'log_conc_uM', 'copyNum', 'totalProtein',
       'totalVolume', 'sample_full_id', 'compound_gene_protein_id', 'row_id',
       'row_gene_id', 'gene_id_dd', 'log_conc_uM_medNorm', 'exp_conc'],
      dtype='object')

In [15]:
conc_df['condition'].unique()

array([nan, 'AD', 'control',
       'LPC: low pathology of plaques and tangles. AD',
       'HPC: high Ab pathology but no detectable cognitive defects. AD',
       'AD: late-stage AD with high pathology scores of plaques and tangles',
       'MCI: mild cognitive impairment with Ab pathology and a slight but measurable defect in cognition. AD',
       'PSP: progressive supranuclear palsy, another neurodegenerative disorder of tauopathy',
       'EAE', 'Alzheimer', 'WT', 'AD_severe', 'AD_intermediate', 'SORT',
       'adult', 'young'], dtype=object)

In [16]:
conc_df = conc_df.loc[(conc_df['condition'].isin(['young', 'adult',  'control',  'WT', 'SORT'])) | (conc_df['condition'].isna()) ]

In [17]:
subsetconc_astrocytes = conc_df[conc_df['location'].isin(['astrocytes'])]

#subsetconc_astrocytes_agg = subsetconc_astrocytes[['Gene names','exp conc']].groupby('Gene names').describe()
subsetconc_astrocytes_agg = subsetconc_astrocytes[['gene_id_final','log_conc_uM_medNorm']].groupby('gene_id_final').describe()



subsetconc_astrocytes_agg.columns = subsetconc_astrocytes_agg.columns.droplevel()

subsetconc_astrocytes_agg = subsetconc_astrocytes_agg.reset_index()

subsetconc_astrocytes_agg.columns =['gene_id_final','count', 'mean', 'std', 'min', '25%', 'median conc uM', '75%', 'max']

subsetconc_astrocytes_agg = subsetconc_astrocytes_agg.drop(columns=['count', 'mean', 'std', 'min', '25%', '75%', 'max'])

subsetconc_astrocytes_agg['gene_id_final'] = subsetconc_astrocytes_agg['gene_id_final'].str.upper()

geneConstraints_astrocytes = pd.merge(subsetconc_astrocytes_agg, geneIdsMapping, how='inner',left_on='gene_id_final',right_on='gene_name')
print(len(geneConstraints_astrocytes))

geneConstraints_astrocytes['median conc uM'] = geneConstraints_astrocytes['median conc uM'].values.astype(float)

908


In [18]:
subsetconc_neurons = conc_df[conc_df['location'].isin(['neurons'])]

#subsetconc_neurons_agg = subsetconc_neurons[['Gene names','exp conc']].groupby('Gene names').describe()
subsetconc_neurons_agg = subsetconc_neurons[['gene_id_final','log_conc_uM_medNorm']].groupby('gene_id_final').describe()


subsetconc_neurons_agg.columns = subsetconc_neurons_agg.columns.droplevel()

subsetconc_neurons_agg = subsetconc_neurons_agg.reset_index()

subsetconc_neurons_agg.columns =['gene_id_final','count', 'mean', 'std', 'min', '25%', 'median conc uM', '75%', 'max']

subsetconc_neurons_agg = subsetconc_neurons_agg.drop(columns=['count', 'mean', 'std', 'min', '25%', '75%', 'max'])

subsetconc_neurons_agg['gene_id_final'] = subsetconc_neurons_agg['gene_id_final'].str.upper()

geneConstraints_neurons = pd.merge(subsetconc_neurons_agg, geneIdsMapping, how='inner',left_on='gene_id_final',right_on='gene_name')
print(len(geneConstraints_neurons))

geneConstraints_neurons['median conc uM'] = geneConstraints_neurons['median conc uM'].values.astype(float)

897


In [19]:
#astrocytes

from sklearn import preprocessing

names = 'median conc uM'
scaler = preprocessing.MinMaxScaler()

scaled_df_astrocytes = scaler.fit_transform(geneConstraints_astrocytes['median conc uM'].values.reshape(-1, 1)) # min is 0, max is 1
geneConstraints_astrocytes['scaled median conc uM'] = scaled_df_astrocytes

# with scaling 

d_geneConstraints_astrocytes = pd.Series(geneConstraints_astrocytes['scaled median conc uM'].values,index=geneConstraints_astrocytes['bigg_id'].astype(str)).to_dict()


In [20]:
#neurons

from sklearn import preprocessing

names = 'median conc uM'
scaler = preprocessing.MinMaxScaler()

scaled_df_neurons = scaler.fit_transform(geneConstraints_neurons['median conc uM'].values.reshape(-1, 1)) # min is 0, max is 1
geneConstraints_neurons['scaled median conc uM'] = scaled_df_neurons

# with scaling 

d_geneConstraints_neurons = pd.Series(geneConstraints_neurons['scaled median conc uM'].values,index=geneConstraints_neurons['bigg_id'].astype(str)).to_dict()


In [21]:
from bioservices.uniprot import UniProt
u = UniProt(verbose=False)

In [22]:
# all genes in neuron, astrocyte,
# cortex, hipp to query uniprot for ec 


len(set(geneConstraints_neurons['gene_id_final'].unique()).union(set(geneConstraints_astrocytes['gene_id_final'].unique())))


g2ec_nach_union = list(set(geneConstraints_neurons['gene_id_final'].unique()).union(set(geneConstraints_astrocytes['gene_id_final'].unique()) ))
len(g2ec_nach_union)

968

In [23]:
gene2ec = dict()

for i,gene in enumerate(g2ec_nach_union):
    searchRes=u.search(gene + '+AND+organism:10090',frmt="tab",columns='protein names',limit=1)
    if ('(EC ' in searchRes):
        searchResFilt=searchRes.split('\n')[1].split('(EC ')[1].split(')')[0]
        gene2ec[gene]=searchResFilt

In [24]:
len(gene2ec)

738

In [25]:
import json

#with open('gene2ec_713_neuron_astrocyte.txt','w') as file:
#    file.write(json.dumps(gene2ec))

with open('gene2ec_neuron_astrocyte_28july2020.txt','w') as file:
    file.write(json.dumps(gene2ec))

In [26]:
# neurons ATP max

# considering AND and OR in gpr

# not so easy !!!!!!! there are complex rules with both and and or

with model:
    for gene_bigg,scaled_conc in d_geneConstraints_neurons.items():  # when scaled, no k_cat # or in d_geneConstraints2_kcat.items() # when k_cat
    #for gene_bigg,scaled_conc in d_geneConstraints .items(): # when k_cat # or in d_geneConstraints2.items() # when scaled, no k_cat
        scaled_conc = scaled_conc*100000.0  # when scaled
        genesreactions = list(model.genes.get_by_id(gene_bigg).reactions)
        for gr in genesreactions:
            #model.reactions.get_by_id(gr.id).bounds = -scaled_conc, scaled_conc
            
            if (model.reactions.get_by_id(gr.id).reversibility == True):
                
                if(model.reactions.get_by_id(gr.id).lower_bound>=0):
                    print('LB problem')  # !!! checked if all lower bounds are below zero and all upper are above zero
                elif(model.reactions.get_by_id(gr.id).upper_bound<=0):
                    print('UB problem') # !!! checked if all lower bounds are below zero and all upper are above zero
                
                elif ( ('or' not in  model.reactions.get_by_id(gr.id).gene_reaction_rule) & ('and' not in model.reactions.get_by_id(gr.id).gene_reaction_rule)):
                    #print(model.reactions.get_by_id(gr.id).gene_reaction_rule)
                    model.reactions.get_by_id(gr.id).bounds = -scaled_conc, scaled_conc
                
                
                elif ( ('or' not in  model.reactions.get_by_id(gr.id).gene_reaction_rule) & ( 'and' in model.reactions.get_by_id(gr.id).gene_reaction_rule) & (model.reactions.get_by_id(gr.id).lower_bound==-100000.0 )& (model.reactions.get_by_id(gr.id).upper_bound==100000.0 )):
                    #print('and          1')
                    model.reactions.get_by_id(gr.id).bounds = -scaled_conc, scaled_conc
                    
                elif ( ('or' not in  model.reactions.get_by_id(gr.id).gene_reaction_rule) & ( 'and' in model.reactions.get_by_id(gr.id).gene_reaction_rule) & (model.reactions.get_by_id(gr.id).lower_bound!=-100000.0 )& (model.reactions.get_by_id(gr.id).upper_bound!=100000.0 )):
                    # !!! checked if all lower bounds are below zero and all upper are above zero
                    
                    model.reactions.get_by_id(gr.id).lower_bound = np.max([model.reactions.get_by_id(gr.id).lower_bound,-scaled_conc]) # max because when we have AND - they are multiple subunits of enzyme, so we take stricter value as capacity boundary 
                    model.reactions.get_by_id(gr.id).upper_bound = np.min([model.reactions.get_by_id(gr.id).upper_bound,scaled_conc]) # same logic as above
                    
                    
                    
                elif ( ( 'or' in model.reactions.get_by_id(gr.id).gene_reaction_rule) & ('and' not in model.reactions.get_by_id(gr.id).gene_reaction_rule) & (model.reactions.get_by_id(gr.id).lower_bound==-100000.0 )& (model.reactions.get_by_id(gr.id).upper_bound==100000.0 )):
                
                    model.reactions.get_by_id(gr.id).bounds = -scaled_conc, scaled_conc
                    
                elif ( ( 'or' in model.reactions.get_by_id(gr.id).gene_reaction_rule) & ('and' not in model.reactions.get_by_id(gr.id).gene_reaction_rule) & (model.reactions.get_by_id(gr.id).lower_bound!=-100000.0 )& (model.reactions.get_by_id(gr.id).upper_bound!=100000.0 )):
                    # !!! checked if all lower bounds are below zero and all upper are above zero
                    
                    model.reactions.get_by_id(gr.id).lower_bound = np.min([model.reactions.get_by_id(gr.id).lower_bound,-scaled_conc]) # min because when we have OR - they are alternatives, wider capacity is more logical here
                    model.reactions.get_by_id(gr.id).upper_bound = np.max([model.reactions.get_by_id(gr.id).upper_bound,scaled_conc]) # same logic as above
                    
                elif ( ('or' in  model.reactions.get_by_id(gr.id).gene_reaction_rule) & ('and' in model.reactions.get_by_id(gr.id).gene_reaction_rule)):
                    gpr = model.reactions.get_by_id(gr.id).gene_reaction_rule
                    #lexems = gpr.replace("(","").replace(")","").replace("and","").replace("or","").split(' ')
                    #lexDict = dict()
                    #for l in lexems:
                    #    if len(l)>0:
                    #        lexDict[l]=d_geneConstraints2_kcat[l]
                    lexems = gpr.split(' or ')
                    
                    #lexList=[]
                    lb1=[]
                    ub1=[]
                    
                    for l in lexems:
                        #print('l',lexems)
                        if ' and ' in l:
                            lexems2 = l.replace("( ","").replace(" )","").split(" and ")
                            mapList =[]
                            #print(lexems2)
                            for l2 in lexems2:
                                if l2 in d_geneConstraints_neurons:
                                    mapList.append(d_geneConstraints_neurons[l2]*100000.0)   # *100000.0 # when scaled
                                
                            lb1.append(np.max(mapList))  ## max for lower bound in AND to have stricter lower bounds based on the limiting subunit of enzyme
                            ub1.append(np.min(mapList))  ## min for upper bound in AND to have stricter upper bounds based on the limiting subunit of enzyme
                             
                        else:
                            if l in d_geneConstraints_neurons:
                                lb1.append(d_geneConstraints_neurons[l]*100000.0) # *100000.0 # when scaled
                                ub1.append(d_geneConstraints_neurons[l]*100000.0) # *100000.0 # when scaled

                    lb1fin = -np.min(lb1)   ###  min for lb in OR to have less strict enzyme out of many isoforms
                    ub1fin = np.max(ub1)   ###  max for ub in OR to have less strict enzyme out of many isoforms
                    
                    model.reactions.get_by_id(gr.id).bounds = lb1fin,ub1fin
                    
                else:
                    print("PROBLEM")
             
            
            
            ############# lb==0 ub!=0
            
            elif( (model.reactions.get_by_id(gr.id).lower_bound==0.0)&(model.reactions.get_by_id(gr.id).upper_bound!=0.0) ):
                if ( ('or' not in  model.reactions.get_by_id(gr.id).gene_reaction_rule) & ('and' not in model.reactions.get_by_id(gr.id).gene_reaction_rule)):
                    model.reactions.get_by_id(gr.id).bounds = 0.0, scaled_conc
                
                elif ( ('or' not in  model.reactions.get_by_id(gr.id).gene_reaction_rule) & ( 'and' in model.reactions.get_by_id(gr.id).gene_reaction_rule) & (model.reactions.get_by_id(gr.id).upper_bound==100000.0 )):
                    model.reactions.get_by_id(gr.id).bounds = 0.0, scaled_conc
                    
                elif ( ('or' not in  model.reactions.get_by_id(gr.id).gene_reaction_rule) & ( 'and' in model.reactions.get_by_id(gr.id).gene_reaction_rule) & (model.reactions.get_by_id(gr.id).upper_bound!=100000.0 )):
                    #model.reactions.get_by_id(gr.id).lower_bound = np.max([model.reactions.get_by_id(gr.id).lower_bound,-scaled_conc]) # max because when we have AND - they are multiple subunits of enzyme, so we take stricter value as capacity boundary 
                    
                    model.reactions.get_by_id(gr.id).upper_bound = np.min([model.reactions.get_by_id(gr.id).upper_bound,scaled_conc]) # same logic as above
                    
                    
                elif ( ( 'or' in model.reactions.get_by_id(gr.id).gene_reaction_rule) & ('and' not in model.reactions.get_by_id(gr.id).gene_reaction_rule) & (model.reactions.get_by_id(gr.id).upper_bound==100000.0)):
                
                    model.reactions.get_by_id(gr.id).bounds = 0.0, scaled_conc
                    
                    
                elif ( ( 'or' in model.reactions.get_by_id(gr.id).gene_reaction_rule) & ('and' not in model.reactions.get_by_id(gr.id).gene_reaction_rule) & (model.reactions.get_by_id(gr.id).upper_bound!=100000.0 )):
                    # !!! checked if all lower bounds are below zero and all upper are above zero
                    
                    model.reactions.get_by_id(gr.id).upper_bound = np.max([model.reactions.get_by_id(gr.id).upper_bound,scaled_conc]) # same logic as above
                     
                    
                elif ( ('or' in  model.reactions.get_by_id(gr.id).gene_reaction_rule) & ('and' in model.reactions.get_by_id(gr.id).gene_reaction_rule)):
                    gpr = model.reactions.get_by_id(gr.id).gene_reaction_rule
                    #lexems = gpr.replace("(","").replace(")","").replace("and","").replace("or","").split(' ')
                    #lexDict = dict()
                    #for l in lexems:
                    #    if len(l)>0:
                    #        lexDict[l]=d_geneConstraints2_kcat[l]
                    lexems = gpr.split(' or ')
                    
                    #lexList=[]
                    lb1=[]
                    ub1=[]
                    
                    for l in lexems:
                        #print('l',lexems)
                        if ' and ' in l:
                            lexems2 = l.replace("( ","").replace(" )","").split(" and ")
                            mapList =[]
                            #print(lexems2)
                            for l2 in lexems2:
                                if l2 in d_geneConstraints_neurons:
                                    mapList.append(d_geneConstraints_neurons[l2]*100000.0) # *100000.0 # when scaled
                                
                            #lb1.append(np.max(mapList))  ## max for lower bound in AND to have stricter lower bounds based on the limiting subunit of enzyme
                            
                            if (len(mapList)>0):
                                ub1.append(np.min(mapList))  ## min for upper bound in AND to have stricter upper bounds based on the limiting subunit of enzyme
                            else:
                                print(lexems2)
                                
                        else:
                            if l in d_geneConstraints_neurons:
                                #lb1.append(d_geneConstraints2_kcat[l])
                                ub1.append(d_geneConstraints_neurons[l]*100000.0) # *100000.0 # when scaled

                    #lb1fin = np.min(lb1)   ###  min for lb in OR to have less strict enzyme out of many isoforms
                    ub1fin = np.max(ub1)   ###  max for ub in OR to have less strict enzyme out of many isoforms
                    model.reactions.get_by_id(gr.id).upper_bound = ub1fin
                
                else:
                    print("PROBLEM")
                    
                
                
            
                
                
                
            ################ lb!=0 ub==0
            
            elif( (model.reactions.get_by_id(gr.id).lower_bound!=0.0)&(model.reactions.get_by_id(gr.id).upper_bound==0.0) ):
                if ( ('or' not in  model.reactions.get_by_id(gr.id).gene_reaction_rule) & ('and' not in model.reactions.get_by_id(gr.id).gene_reaction_rule)):
                    model.reactions.get_by_id(gr.id).bounds = -scaled_conc, 0.0
                
                elif ( ('or' not in  model.reactions.get_by_id(gr.id).gene_reaction_rule) & ( 'and' in model.reactions.get_by_id(gr.id).gene_reaction_rule) & (model.reactions.get_by_id(gr.id).lower_bound==-100000.0 )):
                    model.reactions.get_by_id(gr.id).bounds = -scaled_conc, 0.0
                    
                elif ( ('or' not in  model.reactions.get_by_id(gr.id).gene_reaction_rule) & ( 'and' in model.reactions.get_by_id(gr.id).gene_reaction_rule) & (model.reactions.get_by_id(gr.id).lower_bound!=-100000.0 )):
                    # !!! checked if all lower bounds are below zero and all upper are above zero
                    
                    model.reactions.get_by_id(gr.id).lower_bound = np.max([model.reactions.get_by_id(gr.id).lower_bound,-scaled_conc]) # max because when we have AND - they are multiple subunits of enzyme, so we take stricter value as capacity boundary 
                    
                    
                elif ( ( 'or' in model.reactions.get_by_id(gr.id).gene_reaction_rule) & ('and' not in model.reactions.get_by_id(gr.id).gene_reaction_rule) & (model.reactions.get_by_id(gr.id).lower_bound==-100000.0 )):
                
                    model.reactions.get_by_id(gr.id).bounds = -scaled_conc, 0.0
                    
                elif ( ( 'or' in model.reactions.get_by_id(gr.id).gene_reaction_rule) & ('and' not in model.reactions.get_by_id(gr.id).gene_reaction_rule) & (model.reactions.get_by_id(gr.id).lower_bound!=-100000.0 )):
                    # !!! checked if all lower bounds are below zero and all upper are above zero
                    model.reactions.get_by_id(gr.id).lower_bound = np.min([model.reactions.get_by_id(gr.id).lower_bound,-scaled_conc]) # min because when we have OR - they are alternatives, wider capacity is more logical here
                    
                elif ( ('or' in  model.reactions.get_by_id(gr.id).gene_reaction_rule) & ('and' in model.reactions.get_by_id(gr.id).gene_reaction_rule)):
                    gpr = model.reactions.get_by_id(gr.id).gene_reaction_rule
                    #lexems = gpr.replace("(","").replace(")","").replace("and","").replace("or","").split(' ')
                    #lexDict = dict()
                    #for l in lexems:
                    #    if len(l)>0:
                    #        lexDict[l]=d_geneConstraints2_kcat[l]
                    lexems = gpr.split(' or ')
                    
                    #lexList=[]
                    lb1=[]
                    ub1=[]
                    
                    for l in lexems:
                        #print('l',lexems)
                        if ' and ' in l:
                            lexems2 = l.replace("( ","").replace(" )","").split(" and ")
                            mapList =[]
                            #print(lexems2)
                            for l2 in lexems2:
                                if l2 in d_geneConstraints_neurons:
                                    mapList.append(d_geneConstraints_neurons[l2]*100000.0) # *100000.0 # when scaled
                                
                            lb1.append(np.max(mapList))  ## max for lower bound in AND to have stricter lower bounds based on the limiting subunit of enzyme
                            #ub1.append(np.min(mapList))  ## min for upper bound in AND to have stricter upper bounds based on the limiting subunit of enzyme
                             
                        else:
                            if l in d_geneConstraints_neurons:
                                lb1.append(d_geneConstraints_neurons[l]*100000.0)  # *100000.0 # when scaled
                                #ub1.append(d_geneConstraints2_kcat[l])

                    lb1fin = -np.min(lb1)   ###  min for lb in OR to have less strict enzyme out of many isoforms
                    #ub1fin = np.max(ub1)   ###  max for ub in OR to have less strict enzyme out of many isoforms
                    model.reactions.get_by_id(gr.id).lower_bound = lb1fin
                    
                else:
                    print("PROBLEM")
                
            
            
            ################ lb==0 ub==0
            
            elif( (model.reactions.get_by_id(gr.id).lower_bound==0.0)&(model.reactions.get_by_id(gr.id).upper_bound==0.0) ):
                if ( ('or' not in  model.reactions.get_by_id(gr.id).gene_reaction_rule) & ('and' not in model.reactions.get_by_id(gr.id).gene_reaction_rule)):
                    model.reactions.get_by_id(gr.id).bounds = -scaled_conc, scaled_conc
                
                elif ( ('or' not in  model.reactions.get_by_id(gr.id).gene_reaction_rule) & ( 'and' in model.reactions.get_by_id(gr.id).gene_reaction_rule) & (model.reactions.get_by_id(gr.id).lower_bound==0.0 )& (model.reactions.get_by_id(gr.id).upper_bound==0.0 )):
                    model.reactions.get_by_id(gr.id).bounds = -scaled_conc, scaled_conc
                    
                    
                elif ( ('or' not in  model.reactions.get_by_id(gr.id).gene_reaction_rule) & ( 'and' in model.reactions.get_by_id(gr.id).gene_reaction_rule) & (model.reactions.get_by_id(gr.id).lower_bound!=0.0 )& (model.reactions.get_by_id(gr.id).upper_bound!=0.0 )):
                    ## lb and ub were zeros before previous elif
                    model.reactions.get_by_id(gr.id).lower_bound = np.max([model.reactions.get_by_id(gr.id).lower_bound,-scaled_conc]) # max because when we have AND - they are multiple subunits of enzyme, so we take stricter value as capacity boundary 
                    model.reactions.get_by_id(gr.id).upper_bound = np.min([model.reactions.get_by_id(gr.id).upper_bound,scaled_conc]) # same logic as above
                    
                elif ( ( 'or' in model.reactions.get_by_id(gr.id).gene_reaction_rule) & ('and' not in model.reactions.get_by_id(gr.id).gene_reaction_rule) & (model.reactions.get_by_id(gr.id).lower_bound==0.0 )& (model.reactions.get_by_id(gr.id).upper_bound==0.0 )):
                    model.reactions.get_by_id(gr.id).bounds = -scaled_conc, scaled_conc
                    
                    
                elif ( ( 'or' in model.reactions.get_by_id(gr.id).gene_reaction_rule) & ('and' not in model.reactions.get_by_id(gr.id).gene_reaction_rule) & (model.reactions.get_by_id(gr.id).lower_bound!=0.0 )& (model.reactions.get_by_id(gr.id).upper_bound!=0.0 )):
                    ## lb and ub were zeros before previous elif
                    model.reactions.get_by_id(gr.id).lower_bound = np.min([model.reactions.get_by_id(gr.id).lower_bound,-scaled_conc]) # min because when we have OR - they are alternatives, wider capacity is more logical here
                    model.reactions.get_by_id(gr.id).upper_bound = np.max([model.reactions.get_by_id(gr.id).upper_bound,scaled_conc]) # same logic as above
                    
                elif ( ('or' in  model.reactions.get_by_id(gr.id).gene_reaction_rule) & ('and' in model.reactions.get_by_id(gr.id).gene_reaction_rule)):
                    gpr = model.reactions.get_by_id(gr.id).gene_reaction_rule
                    #lexems = gpr.replace("(","").replace(")","").replace("and","").replace("or","").split(' ')
                    #lexDict = dict()
                    #for l in lexems:
                    #    if len(l)>0:
                    #        lexDict[l]=d_geneConstraints2_kcat[l]
                    lexems = gpr.split(' or ')
                    
                    #lexList=[]
                    lb1=[]
                    ub1=[]
                    
                    for l in lexems:
                        #print('l',lexems)
                        if ' and ' in l:
                            lexems2 = l.replace("( ","").replace(" )","").split(" and ")
                            mapList =[]
                            #print(lexems2)
                            for l2 in lexems2:
                                if l2 in d_geneConstraints_neurons:
                                    mapList.append(d_geneConstraints_neurons[l2]*100000.0)  # *100000.0 # when scaled
                                
                            lb1.append(np.max(mapList))  ## max for lower bound in AND to have stricter lower bounds based on the limiting subunit of enzyme
                            ub1.append(np.min(mapList))  ## min for upper bound in AND to have stricter upper bounds based on the limiting subunit of enzyme
                             
                        else:
                            if l in d_geneConstraints_neurons:
                                lb1.append(d_geneConstraints_neurons[l]*100000.0)  # *100000.0 # when scaled
                                ub1.append(d_geneConstraints_neurons[l]*100000.0)  # *100000.0 # when scaled

                    lb1fin = -np.min(lb1)   ###  min for lb in OR to have less strict enzyme out of many isoforms
                    ub1fin = np.max(ub1)   ###  max for ub in OR to have less strict enzyme out of many isoforms
                    model.reactions.get_by_id(gr.id).bounds = lb1fin,ub1fin
                else:
                    print("PROBLEM")
                
                
                
                
                
            #######################
            else:
                print("PROBLEM ### ",model.reactions.get_by_id(gr.id).id, 'gpr: ', model.reactions.get_by_id(gr.id).gene_reaction_rule)
                
                
                
    
    model.objective = {
        #included below model.reactions.get_by_id("ATPS4m"): 1, # adp_m + 4.0 h_c + pi_m --> atp_m + h2o_m + 3.0 h_m ATP synthase (four protons for one ATP)
        model.reactions.get_by_id("AP4AH1"): 1,  # tried without this reaction
       #included below  model.reactions.get_by_id("PYK"): 1,
        
       model.reactions.get_by_id("NDPK1m"): -1, # tried without this reaction
        #model.reactions.get_by_id("HEX1"): -1,
        #model.reactions.get_by_id("PFK"): -1,
       #included below model.reactions.get_by_id("PGK"): -1,
        
       model.reactions.get_by_id("SUCOAS1m"): -1, # to max gtp too
        
      #included below  model.reactions.get_by_id("ADNCYC"): -1, # adenylatecyclase
        model.reactions.get_by_id("SUCD1m"): 1, # SDH
        
        
        model.reactions.get_by_id("ATPS4m"): 1,
        model.reactions.get_by_id("ATPtm"): 1,
        model.reactions.get_by_id("AP4AH1"): 1,
     #   model.reactions.get_by_id("DNDPt56m"): 1,
     #   model.reactions.get_by_id("DNDPt57m"): 1,
     #   model.reactions.get_by_id("DNDPt19m"): 1,
     #   model.reactions.get_by_id("DNDPt44m"): 1,
     #   model.reactions.get_by_id("DNDPt43m"): 1,
     #   model.reactions.get_by_id("DNDPt20m"): 1,
        model.reactions.get_by_id("PYK"): 1
     #   model.reactions.get_by_id("DNDPt32m"): 1,
     #   model.reactions.get_by_id("DNDPt2m"): 1,
     #   model.reactions.get_by_id("DNDPt13m"): 1,
     #   model.reactions.get_by_id("DNDPt31m"): 1
        
        
     #   model.reactions.get_by_id('BIOMASS_mm_1_no_glygln'): 1 ### biomass added to balance 
    }
    
    ## !!! try to set redox state as a part of the obj fun
    
    model.solution = model.optimize()
    result_neurons_fba = model.solution.to_frame()
    print(model.solution.status, '\t',  model.solution, '\t', model.objective)
    result_neurons = flux_variability_analysis(model,fraction_of_optimum=0.9,loopless=True)
      
    
   

['16770', '14595']
['8908', '232493']
['232493', '27357']
['232493', '27357']
['8908', '232493']
['18587', '225600', '78600']
['18587', '225600', '78600']
['18587', '225600', '78600']
['18587', '225600', '78600']
['18587', '225600', '78600']
['18587', '225600', '78600']
['18587', '225600', '78600']
['18587', '225600', '78600']
['18587', '225600', '78600']
['20532', '30962']
['20532', '30962']
optimal 	 <Solution 243953.770 at 0x7fb2f45ec438> 	 Maximize
1.0*AP4AH1 - 1.0*AP4AH1_reverse_9176b + 1.0*ATPS4m - 1.0*ATPS4m_reverse_d370c + 1.0*ATPtm - 1.0*ATPtm_reverse_b45b1 - 1.0*NDPK1m + 1.0*NDPK1m_reverse_b52dc + 1.0*PYK - 1.0*PYK_reverse_bc8ff + 1.0*SUCD1m - 1.0*SUCD1m_reverse_60ba1 - 1.0*SUCOAS1m + 1.0*SUCOAS1m_reverse_927c1


In [27]:
# astrocytes ATP max

# considering AND and OR in gpr

# not so easy !!!!!!! there are complex rules with both and and or

with model:
    for gene_bigg,scaled_conc in d_geneConstraints_astrocytes.items():  # when scaled, no k_cat # or in d_geneConstraints2_kcat.items() # when k_cat
    #for gene_bigg,scaled_conc in d_geneConstraints .items(): # when k_cat # or in d_geneConstraints2.items() # when scaled, no k_cat
        scaled_conc = scaled_conc*100000.0  # when scaled
        genesreactions = list(model.genes.get_by_id(gene_bigg).reactions)
        for gr in genesreactions:
            #model.reactions.get_by_id(gr.id).bounds = -scaled_conc, scaled_conc
            
            if (model.reactions.get_by_id(gr.id).reversibility == True):
                
                if(model.reactions.get_by_id(gr.id).lower_bound>=0):
                    print('LB problem')  # !!! checked if all lower bounds are below zero and all upper are above zero
                elif(model.reactions.get_by_id(gr.id).upper_bound<=0):
                    print('UB problem') # !!! checked if all lower bounds are below zero and all upper are above zero
                
                elif ( ('or' not in  model.reactions.get_by_id(gr.id).gene_reaction_rule) & ('and' not in model.reactions.get_by_id(gr.id).gene_reaction_rule)):
                    #print(model.reactions.get_by_id(gr.id).gene_reaction_rule)
                    model.reactions.get_by_id(gr.id).bounds = -scaled_conc, scaled_conc
                
                
                elif ( ('or' not in  model.reactions.get_by_id(gr.id).gene_reaction_rule) & ( 'and' in model.reactions.get_by_id(gr.id).gene_reaction_rule) & (model.reactions.get_by_id(gr.id).lower_bound==-100000.0 )& (model.reactions.get_by_id(gr.id).upper_bound==100000.0 )):
                    #print('and          1')
                    model.reactions.get_by_id(gr.id).bounds = -scaled_conc, scaled_conc
                    
                elif ( ('or' not in  model.reactions.get_by_id(gr.id).gene_reaction_rule) & ( 'and' in model.reactions.get_by_id(gr.id).gene_reaction_rule) & (model.reactions.get_by_id(gr.id).lower_bound!=-100000.0 )& (model.reactions.get_by_id(gr.id).upper_bound!=100000.0 )):
                    # !!! checked if all lower bounds are below zero and all upper are above zero
                    
                    model.reactions.get_by_id(gr.id).lower_bound = np.max([model.reactions.get_by_id(gr.id).lower_bound,-scaled_conc]) # max because when we have AND - they are multiple subunits of enzyme, so we take stricter value as capacity boundary 
                    model.reactions.get_by_id(gr.id).upper_bound = np.min([model.reactions.get_by_id(gr.id).upper_bound,scaled_conc]) # same logic as above
                    
                    
                    
                elif ( ( 'or' in model.reactions.get_by_id(gr.id).gene_reaction_rule) & ('and' not in model.reactions.get_by_id(gr.id).gene_reaction_rule) & (model.reactions.get_by_id(gr.id).lower_bound==-100000.0 )& (model.reactions.get_by_id(gr.id).upper_bound==100000.0 )):
                
                    model.reactions.get_by_id(gr.id).bounds = -scaled_conc, scaled_conc
                    
                elif ( ( 'or' in model.reactions.get_by_id(gr.id).gene_reaction_rule) & ('and' not in model.reactions.get_by_id(gr.id).gene_reaction_rule) & (model.reactions.get_by_id(gr.id).lower_bound!=-100000.0 )& (model.reactions.get_by_id(gr.id).upper_bound!=100000.0 )):
                    # !!! checked if all lower bounds are below zero and all upper are above zero
                    
                    model.reactions.get_by_id(gr.id).lower_bound = np.min([model.reactions.get_by_id(gr.id).lower_bound,-scaled_conc]) # min because when we have OR - they are alternatives, wider capacity is more logical here
                    model.reactions.get_by_id(gr.id).upper_bound = np.max([model.reactions.get_by_id(gr.id).upper_bound,scaled_conc]) # same logic as above
                    
                elif ( ('or' in  model.reactions.get_by_id(gr.id).gene_reaction_rule) & ('and' in model.reactions.get_by_id(gr.id).gene_reaction_rule)):
                    gpr = model.reactions.get_by_id(gr.id).gene_reaction_rule
                    #lexems = gpr.replace("(","").replace(")","").replace("and","").replace("or","").split(' ')
                    #lexDict = dict()
                    #for l in lexems:
                    #    if len(l)>0:
                    #        lexDict[l]=d_geneConstraints2_kcat[l]
                    lexems = gpr.split(' or ')
                    
                    #lexList=[]
                    lb1=[]
                    ub1=[]
                    
                    for l in lexems:
                        #print('l',lexems)
                        if ' and ' in l:
                            lexems2 = l.replace("( ","").replace(" )","").split(" and ")
                            mapList =[]
                            #print(lexems2)
                            for l2 in lexems2:
                                if l2 in d_geneConstraints_astrocytes:
                                    mapList.append(d_geneConstraints_astrocytes[l2]*100000.0)   # *100000.0 # when scaled
                                
                            lb1.append(np.max(mapList))  ## max for lower bound in AND to have stricter lower bounds based on the limiting subunit of enzyme
                            ub1.append(np.min(mapList))  ## min for upper bound in AND to have stricter upper bounds based on the limiting subunit of enzyme
                             
                        else:
                            if l in d_geneConstraints_astrocytes:
                                lb1.append(d_geneConstraints_astrocytes[l]*100000.0) # *100000.0 # when scaled
                                ub1.append(d_geneConstraints_astrocytes[l]*100000.0) # *100000.0 # when scaled

                    lb1fin = -np.min(lb1)   ###  min for lb in OR to have less strict enzyme out of many isoforms
                    ub1fin = np.max(ub1)   ###  max for ub in OR to have less strict enzyme out of many isoforms
                    
                    model.reactions.get_by_id(gr.id).bounds = lb1fin,ub1fin
                    
                else:
                    print("PROBLEM")
             
            
            
            ############# lb==0 ub!=0
            
            elif( (model.reactions.get_by_id(gr.id).lower_bound==0.0)&(model.reactions.get_by_id(gr.id).upper_bound!=0.0) ):
                if ( ('or' not in  model.reactions.get_by_id(gr.id).gene_reaction_rule) & ('and' not in model.reactions.get_by_id(gr.id).gene_reaction_rule)):
                    model.reactions.get_by_id(gr.id).bounds = 0.0, scaled_conc
                
                elif ( ('or' not in  model.reactions.get_by_id(gr.id).gene_reaction_rule) & ( 'and' in model.reactions.get_by_id(gr.id).gene_reaction_rule) & (model.reactions.get_by_id(gr.id).upper_bound==100000.0 )):
                    model.reactions.get_by_id(gr.id).bounds = 0.0, scaled_conc
                    
                elif ( ('or' not in  model.reactions.get_by_id(gr.id).gene_reaction_rule) & ( 'and' in model.reactions.get_by_id(gr.id).gene_reaction_rule) & (model.reactions.get_by_id(gr.id).upper_bound!=100000.0 )):
                    #model.reactions.get_by_id(gr.id).lower_bound = np.max([model.reactions.get_by_id(gr.id).lower_bound,-scaled_conc]) # max because when we have AND - they are multiple subunits of enzyme, so we take stricter value as capacity boundary 
                    
                    model.reactions.get_by_id(gr.id).upper_bound = np.min([model.reactions.get_by_id(gr.id).upper_bound,scaled_conc]) # same logic as above
                    
                    
                elif ( ( 'or' in model.reactions.get_by_id(gr.id).gene_reaction_rule) & ('and' not in model.reactions.get_by_id(gr.id).gene_reaction_rule) & (model.reactions.get_by_id(gr.id).upper_bound==100000.0)):
                
                    model.reactions.get_by_id(gr.id).bounds = 0.0, scaled_conc
                    
                    
                elif ( ( 'or' in model.reactions.get_by_id(gr.id).gene_reaction_rule) & ('and' not in model.reactions.get_by_id(gr.id).gene_reaction_rule) & (model.reactions.get_by_id(gr.id).upper_bound!=100000.0 )):
                    # !!! checked if all lower bounds are below zero and all upper are above zero
                    
                    model.reactions.get_by_id(gr.id).upper_bound = np.max([model.reactions.get_by_id(gr.id).upper_bound,scaled_conc]) # same logic as above
                     
                    
                elif ( ('or' in  model.reactions.get_by_id(gr.id).gene_reaction_rule) & ('and' in model.reactions.get_by_id(gr.id).gene_reaction_rule)):
                    gpr = model.reactions.get_by_id(gr.id).gene_reaction_rule
                    #lexems = gpr.replace("(","").replace(")","").replace("and","").replace("or","").split(' ')
                    #lexDict = dict()
                    #for l in lexems:
                    #    if len(l)>0:
                    #        lexDict[l]=d_geneConstraints2_kcat[l]
                    lexems = gpr.split(' or ')
                    
                    #lexList=[]
                    lb1=[]
                    ub1=[]
                    
                    for l in lexems:
                        #print('l',lexems)
                        if ' and ' in l:
                            lexems2 = l.replace("( ","").replace(" )","").split(" and ")
                            mapList =[]
                            #print(lexems2)
                            for l2 in lexems2:
                                if l2 in d_geneConstraints_astrocytes:
                                    mapList.append(d_geneConstraints_astrocytes[l2]*100000.0) # *100000.0 # when scaled
                                
                            #lb1.append(np.max(mapList))  ## max for lower bound in AND to have stricter lower bounds based on the limiting subunit of enzyme
                            
                            if (len(mapList)>0):
                                ub1.append(np.min(mapList))  ## min for upper bound in AND to have stricter upper bounds based on the limiting subunit of enzyme
                            else:
                                print(lexems2)
                                
                        else:
                            if l in d_geneConstraints_astrocytes:
                                #lb1.append(d_geneConstraints2_kcat[l])
                                ub1.append(d_geneConstraints_astrocytes[l]*100000.0) # *100000.0 # when scaled

                    #lb1fin = np.min(lb1)   ###  min for lb in OR to have less strict enzyme out of many isoforms
                    ub1fin = np.max(ub1)   ###  max for ub in OR to have less strict enzyme out of many isoforms
                    model.reactions.get_by_id(gr.id).upper_bound = ub1fin
                
                else:
                    print("PROBLEM")
                    
                
                
            
                
                
                
            ################ lb!=0 ub==0
            
            elif( (model.reactions.get_by_id(gr.id).lower_bound!=0.0)&(model.reactions.get_by_id(gr.id).upper_bound==0.0) ):
                if ( ('or' not in  model.reactions.get_by_id(gr.id).gene_reaction_rule) & ('and' not in model.reactions.get_by_id(gr.id).gene_reaction_rule)):
                    model.reactions.get_by_id(gr.id).bounds = -scaled_conc, 0.0
                
                elif ( ('or' not in  model.reactions.get_by_id(gr.id).gene_reaction_rule) & ( 'and' in model.reactions.get_by_id(gr.id).gene_reaction_rule) & (model.reactions.get_by_id(gr.id).lower_bound==-100000.0 )):
                    model.reactions.get_by_id(gr.id).bounds = -scaled_conc, 0.0
                    
                elif ( ('or' not in  model.reactions.get_by_id(gr.id).gene_reaction_rule) & ( 'and' in model.reactions.get_by_id(gr.id).gene_reaction_rule) & (model.reactions.get_by_id(gr.id).lower_bound!=-100000.0 )):
                    # !!! checked if all lower bounds are below zero and all upper are above zero
                    
                    model.reactions.get_by_id(gr.id).lower_bound = np.max([model.reactions.get_by_id(gr.id).lower_bound,-scaled_conc]) # max because when we have AND - they are multiple subunits of enzyme, so we take stricter value as capacity boundary 
                    
                    
                elif ( ( 'or' in model.reactions.get_by_id(gr.id).gene_reaction_rule) & ('and' not in model.reactions.get_by_id(gr.id).gene_reaction_rule) & (model.reactions.get_by_id(gr.id).lower_bound==-100000.0 )):
                
                    model.reactions.get_by_id(gr.id).bounds = -scaled_conc, 0.0
                    
                elif ( ( 'or' in model.reactions.get_by_id(gr.id).gene_reaction_rule) & ('and' not in model.reactions.get_by_id(gr.id).gene_reaction_rule) & (model.reactions.get_by_id(gr.id).lower_bound!=-100000.0 )):
                    # !!! checked if all lower bounds are below zero and all upper are above zero
                    model.reactions.get_by_id(gr.id).lower_bound = np.min([model.reactions.get_by_id(gr.id).lower_bound,-scaled_conc]) # min because when we have OR - they are alternatives, wider capacity is more logical here
                    
                elif ( ('or' in  model.reactions.get_by_id(gr.id).gene_reaction_rule) & ('and' in model.reactions.get_by_id(gr.id).gene_reaction_rule)):
                    gpr = model.reactions.get_by_id(gr.id).gene_reaction_rule
                    #lexems = gpr.replace("(","").replace(")","").replace("and","").replace("or","").split(' ')
                    #lexDict = dict()
                    #for l in lexems:
                    #    if len(l)>0:
                    #        lexDict[l]=d_geneConstraints2_kcat[l]
                    lexems = gpr.split(' or ')
                    
                    #lexList=[]
                    lb1=[]
                    ub1=[]
                    
                    for l in lexems:
                        #print('l',lexems)
                        if ' and ' in l:
                            lexems2 = l.replace("( ","").replace(" )","").split(" and ")
                            mapList =[]
                            #print(lexems2)
                            for l2 in lexems2:
                                if l2 in d_geneConstraints_astrocytes:
                                    mapList.append(d_geneConstraints_astrocytes[l2]*100000.0) # *100000.0 # when scaled
                                
                            lb1.append(np.max(mapList))  ## max for lower bound in AND to have stricter lower bounds based on the limiting subunit of enzyme
                            #ub1.append(np.min(mapList))  ## min for upper bound in AND to have stricter upper bounds based on the limiting subunit of enzyme
                             
                        else:
                            if l in d_geneConstraints_astrocytes:
                                lb1.append(d_geneConstraints_astrocytes[l]*100000.0)  # *100000.0 # when scaled
                                #ub1.append(d_geneConstraints2_kcat[l])

                    lb1fin = -np.min(lb1)   ###  min for lb in OR to have less strict enzyme out of many isoforms
                    #ub1fin = np.max(ub1)   ###  max for ub in OR to have less strict enzyme out of many isoforms
                    model.reactions.get_by_id(gr.id).lower_bound = lb1fin
                    
                else:
                    print("PROBLEM")
                
            
            
            ################ lb==0 ub==0
            
            elif( (model.reactions.get_by_id(gr.id).lower_bound==0.0)&(model.reactions.get_by_id(gr.id).upper_bound==0.0) ):
                if ( ('or' not in  model.reactions.get_by_id(gr.id).gene_reaction_rule) & ('and' not in model.reactions.get_by_id(gr.id).gene_reaction_rule)):
                    model.reactions.get_by_id(gr.id).bounds = -scaled_conc, scaled_conc
                
                elif ( ('or' not in  model.reactions.get_by_id(gr.id).gene_reaction_rule) & ( 'and' in model.reactions.get_by_id(gr.id).gene_reaction_rule) & (model.reactions.get_by_id(gr.id).lower_bound==0.0 )& (model.reactions.get_by_id(gr.id).upper_bound==0.0 )):
                    model.reactions.get_by_id(gr.id).bounds = -scaled_conc, scaled_conc
                    
                    
                elif ( ('or' not in  model.reactions.get_by_id(gr.id).gene_reaction_rule) & ( 'and' in model.reactions.get_by_id(gr.id).gene_reaction_rule) & (model.reactions.get_by_id(gr.id).lower_bound!=0.0 )& (model.reactions.get_by_id(gr.id).upper_bound!=0.0 )):
                    ## lb and ub were zeros before previous elif
                    model.reactions.get_by_id(gr.id).lower_bound = np.max([model.reactions.get_by_id(gr.id).lower_bound,-scaled_conc]) # max because when we have AND - they are multiple subunits of enzyme, so we take stricter value as capacity boundary 
                    model.reactions.get_by_id(gr.id).upper_bound = np.min([model.reactions.get_by_id(gr.id).upper_bound,scaled_conc]) # same logic as above
                    
                elif ( ( 'or' in model.reactions.get_by_id(gr.id).gene_reaction_rule) & ('and' not in model.reactions.get_by_id(gr.id).gene_reaction_rule) & (model.reactions.get_by_id(gr.id).lower_bound==0.0 )& (model.reactions.get_by_id(gr.id).upper_bound==0.0 )):
                    model.reactions.get_by_id(gr.id).bounds = -scaled_conc, scaled_conc
                    
                    
                elif ( ( 'or' in model.reactions.get_by_id(gr.id).gene_reaction_rule) & ('and' not in model.reactions.get_by_id(gr.id).gene_reaction_rule) & (model.reactions.get_by_id(gr.id).lower_bound!=0.0 )& (model.reactions.get_by_id(gr.id).upper_bound!=0.0 )):
                    ## lb and ub were zeros before previous elif
                    model.reactions.get_by_id(gr.id).lower_bound = np.min([model.reactions.get_by_id(gr.id).lower_bound,-scaled_conc]) # min because when we have OR - they are alternatives, wider capacity is more logical here
                    model.reactions.get_by_id(gr.id).upper_bound = np.max([model.reactions.get_by_id(gr.id).upper_bound,scaled_conc]) # same logic as above
                    
                elif ( ('or' in  model.reactions.get_by_id(gr.id).gene_reaction_rule) & ('and' in model.reactions.get_by_id(gr.id).gene_reaction_rule)):
                    gpr = model.reactions.get_by_id(gr.id).gene_reaction_rule
                    #lexems = gpr.replace("(","").replace(")","").replace("and","").replace("or","").split(' ')
                    #lexDict = dict()
                    #for l in lexems:
                    #    if len(l)>0:
                    #        lexDict[l]=d_geneConstraints2_kcat[l]
                    lexems = gpr.split(' or ')
                    
                    #lexList=[]
                    lb1=[]
                    ub1=[]
                    
                    for l in lexems:
                        #print('l',lexems)
                        if ' and ' in l:
                            lexems2 = l.replace("( ","").replace(" )","").split(" and ")
                            mapList =[]
                            #print(lexems2)
                            for l2 in lexems2:
                                if l2 in d_geneConstraints_astrocytes:
                                    mapList.append(d_geneConstraints_astrocytes[l2]*100000.0)  # *100000.0 # when scaled
                                
                            lb1.append(np.max(mapList))  ## max for lower bound in AND to have stricter lower bounds based on the limiting subunit of enzyme
                            ub1.append(np.min(mapList))  ## min for upper bound in AND to have stricter upper bounds based on the limiting subunit of enzyme
                             
                        else:
                            if l in d_geneConstraints_astrocytes:
                                lb1.append(d_geneConstraints_astrocytes[l]*100000.0)  # *100000.0 # when scaled
                                ub1.append(d_geneConstraints_astrocytes[l]*100000.0)  # *100000.0 # when scaled

                    lb1fin = -np.min(lb1)   ###  min for lb in OR to have less strict enzyme out of many isoforms
                    ub1fin = np.max(ub1)   ###  max for ub in OR to have less strict enzyme out of many isoforms
                    model.reactions.get_by_id(gr.id).bounds = lb1fin,ub1fin
                else:
                    print("PROBLEM")
                
                
                
                
                
            #######################
            else:
                print("PROBLEM ### ",model.reactions.get_by_id(gr.id).id, 'gpr: ', model.reactions.get_by_id(gr.id).gene_reaction_rule)
                
                
                
    
    model.objective = {
        #included below model.reactions.get_by_id("ATPS4m"): 1, # adp_m + 4.0 h_c + pi_m --> atp_m + h2o_m + 3.0 h_m ATP synthase (four protons for one ATP)
        model.reactions.get_by_id("AP4AH1"): 1,  # tried without this reaction
       #included below  model.reactions.get_by_id("PYK"): 1,
        
       model.reactions.get_by_id("NDPK1m"): -1, # tried without this reaction
        #model.reactions.get_by_id("HEX1"): -1,
        #model.reactions.get_by_id("PFK"): -1,
       #included below model.reactions.get_by_id("PGK"): -1,
        
       model.reactions.get_by_id("SUCOAS1m"): -1, # to max gtp too
        
      #included below  model.reactions.get_by_id("ADNCYC"): -1, # adenylatecyclase
        model.reactions.get_by_id("SUCD1m"): 1, # SDH
        
        
        model.reactions.get_by_id("ATPS4m"): 1,
        model.reactions.get_by_id("ATPtm"): 1,
        model.reactions.get_by_id("AP4AH1"): 1,
     #   model.reactions.get_by_id("DNDPt56m"): 1,
     #   model.reactions.get_by_id("DNDPt57m"): 1,
     #   model.reactions.get_by_id("DNDPt19m"): 1,
     #   model.reactions.get_by_id("DNDPt44m"): 1,
     #   model.reactions.get_by_id("DNDPt43m"): 1,
     #   model.reactions.get_by_id("DNDPt20m"): 1,
        model.reactions.get_by_id("PYK"): 1
     #   model.reactions.get_by_id("DNDPt32m"): 1,
     #   model.reactions.get_by_id("DNDPt2m"): 1,
     #   model.reactions.get_by_id("DNDPt13m"): 1,
     #   model.reactions.get_by_id("DNDPt31m"): 1
        
        
     #   model.reactions.get_by_id('BIOMASS_mm_1_no_glygln'): 1 ### biomass added to balance 
    }
    
    ## !!! try to set redox state as a part of the obj fun
    
    model.solution = model.optimize()
    result_astrocytes_fba = model.solution.to_frame()
    print(model.solution.status, '\t',  model.solution, '\t', model.objective)
    result_astrocytes = flux_variability_analysis(model,fraction_of_optimum=0.9,loopless=True)
      
    
   

['20532', '30962']
['20532', '30962']
['8908', '232493']
['8908', '232493']
['8908', '232493']
['8908', '232493']
['18587', '225600', '78600']
['18587', '225600', '78600']
['18587', '225600', '78600']
['18587', '225600', '78600']
['18587', '225600', '78600']
['18587', '225600', '78600']
['18587', '225600', '78600']
['18587', '225600', '78600']
['20532', '30962']
['20532', '30962']
optimal 	 <Solution 264967.620 at 0x7fb2f45e3c88> 	 Maximize
1.0*AP4AH1 - 1.0*AP4AH1_reverse_9176b + 1.0*ATPS4m - 1.0*ATPS4m_reverse_d370c + 1.0*ATPtm - 1.0*ATPtm_reverse_b45b1 - 1.0*NDPK1m + 1.0*NDPK1m_reverse_b52dc + 1.0*PYK - 1.0*PYK_reverse_bc8ff + 1.0*SUCD1m - 1.0*SUCD1m_reverse_60ba1 - 1.0*SUCOAS1m + 1.0*SUCOAS1m_reverse_927c1


In [28]:
result_na_max = (result_neurons.maximum -result_astrocytes.maximum).to_dict()
map_result_na_max = escher.Builder('RECON1.Glycolysis TCA PPP', reaction_data=result_na_max)
#map_abs_flux_max_90optIniModATPobj_max.save_html('looplessFVA_max_iniModelATPObj_90opt_max.html') # this was for V1

map_result_na_max.display_in_notebook()

In [29]:
map_result_na_max.save_html('map_result_neuronsmax_minus_astrocytesmax_28july2020.html')



'map_result_neuronsmax_minus_astrocytesmax_28july2020.html'